In [ ]:
import random
from tqdm import tqdm
import torch.nn as nn
import torch
from transformers import GPT2LMHeadModel, GPT2Config
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.model_selection import train_test_split
from transformers import  AutoConfig, AutoModelWithLMHead
from torch.utils.data import TensorDataset
from nltk.translate.bleu_score import corpus_bleu

# آ )
<div dir=rtl>

پیش پردازش دادگان

In [ ]:
data_file_path = '/content/ferdousi.txt'

with open(data_file_path, 'r', encoding='utf-8') as file:
    lines = file.read()

lines=lines.splitlines()
del lines[99218]
del lines[0]
del lines[0]

input_lines = lines[::2] #even
output_lines = lines[1::2] #odd

train_input,test_in,train_output,test_out= train_test_split(input_lines,output_lines, test_size=0.2, random_state=42)



In [ ]:
train = [train_input[i] + '    ' +train_output[i] for i in range(0, len(train_input),1)]

test_input=[test_in[i]+'  ' for i in range(0, len(test_input),1)]
test_output=[test_in[i] +'    ' +test_out[i] for i in range(0, len(test_input),1)]

# ب

In [ ]:
max_length_characters = max(len(poem) for poem in poem)

In [ ]:
max_length_characters

65

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/gpt2-fa")#, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

train_input_tokenized = tokenizer(train, return_tensors='pt', padding=True, truncation=True, max_length=65)
test_input_tokenized = tokenizer(test_input, return_tensors='pt', padding=True, truncation=True, max_length=65)
test_output_tokenized = tokenizer(test_output, return_tensors='pt', padding=True, truncation=True, max_length=65)



train_ids = torch.tensor(train_input_tokenized['input_ids'].squeeze())
train_attention = torch.tensor(train_input_tokenized['attention_mask'])
test_ids = torch.tensor(test_input_tokenized['input_ids'])
test_labels = torch.tensor(test_output_tokenized['input_ids'])
test_attention = torch.tensor(test_input_tokenized['attention_mask'])

train_dataset = TensorDataset(train_ids, train_ids,train_attention)
test_dataset = TensorDataset(test_ids, test_labels ,test_attention)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

<ipython-input-25-37126281f117>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_ids = torch.tensor(train_input_tokenized['input_ids'].squeeze())
<ipython-input-25-37126281f117>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_attention = torch.tensor(train_input_tokenized['attention_mask'])
<ipython-input-25-37126281f117>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_ids = torch.tensor(test_input_tokenized['input_ids'])
<ipython-input-25-37126281f117>:25: UserWarning: To copy construct from a tensor, it is recommen

# ج

In [ ]:

configuration = GPT2Config.from_pretrained('HooshvareLab/gpt2-fa')
model = AutoModelForCausalLM.from_pretrained('HooshvareLab/gpt2-fa')
model.config.pad_token_id = model.config.eos_token_id


pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

In [ ]:
epochs = 15
learning_rate = 0.0001
warmup_steps = 1e2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=len(train_dataloader) * epochs)
criterion = nn.CrossEntropyLoss()

for epoch_i in tqdm(range(0, epochs), position=0):

    print(f'Beginning epoch {epoch_i + 1} of {epochs}')
    total_train_loss = 0

    model.train()
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):
        b_input_ids = batch[0].to(device).squeeze()
        b_labels = batch[1].to(device).squeeze()
        b_masks = batch[2].to(device).squeeze()
        b_labels = b_labels.clone()

        model.zero_grad()

        outputs = model(b_input_ids, attention_mask=b_masks, labels=b_labels)

        loss=outputs.loss
        batch_loss = loss.item()
        total_train_loss += batch_loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f'\nEpoch {epoch_i + 1} - Training Loss: {avg_train_loss:.4f}')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/15 [00:00<?, ?it/s]

Beginning epoch 1 of 15


  7%|▋         | 1/15 [04:02<56:37, 242.71s/it]


Epoch 1 - Training Loss: 3.4076
Beginning epoch 2 of 15


 13%|█▎        | 2/15 [08:13<53:35, 247.36s/it]

...


Epoch 13 - Training Loss: 0.8795
Beginning epoch 14 of 15


 93%|█████████▎| 14/15 [58:42<04:12, 252.24s/it]


Epoch 14 - Training Loss: 0.8354
Beginning epoch 15 of 15


100%|██████████| 15/15 [1:02:54<00:00, 251.65s/it]


Epoch 15 - Training Loss: 0.8025


# د

In [ ]:
total=0
total_test_loss=0
t_bleu=0
with torch.no_grad():
    for step, batch in enumerate(test_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)
        b_masks = batch[2].to(device)
        b_labels = b_labels.clone()


        output = model.generate(
        b_input_ids,
        max_length=25,
        num_beams=20,
        no_repeat_ngram_size=2,
        top_k=30,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=torch.ones_like(b_input_ids),
        do_sample=True
        )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        reference_text = tokenizer.decode(b_labels[0], skip_special_tokens=True)
        print(generated_text)
        print(reference_text)

        bleu_score = corpus_bleu([[reference_text.split()]], [generated_text.split()])
        print(bleu_score)
        t_bleu += bleu_score


average_bleu_score = t_bleu / len(test_dataloader)
print(f'average_bleu_score: {average_bleu_score:.4f}')


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


گنهکارتر چیز مردم بود   همیشه به کام دل مرد نیکی بود
گنهکارتر چیز مردم بود    که از کین و آزش خرد گم بود
0.22652581507881553


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


بفرمود تا رفت نزدیک تخت   نشست از بر جامهٔ شاه نیک بخت
بفرمود تا رفت نزدیک تخت    دل شاه کسری غمی گشت سخت
0.33180774028439425


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

فرومایه ضحاک بیدادگر   ز گیتی نیابی و ز تخت و کمر
فرومایه ضحاک بیدادگر    بدین چاره بگرفت جای پدر
3.4077295621678842e-78


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


و دیگر که دارنده یار منست   سر اختر اندر کنار من افسرست
و دیگر که دارنده یار منست    بزرگی و مهرش حصار منست
0.4172261448611506


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


دگر هفته چون هور بفراخت تاج   نشست از بر تخت پیروز با تخت عاج
دگر هفته چون هور بفراخت تاج    بیامد نشست از بر تخت عاج
0.5607900239988007
بدو سرشبان گفت کای شهریار   دلت را بدین کار با من مدار
بدو سرشبان گفت کای شهریار    ز گیتی من آیم بدین مرغزار
0.3448444257953326
average_bleu_score: 0.3718


In [ ]:
def generate_poem(poem_stanza):
    model.eval()

    input_tokenized = tokenizer(poem_stanza, return_tensors='pt', padding=True, truncation=True, max_length=35)
    input_ids = torch.tensor(input_tokenized['input_ids']).to(device)

    output = model.generate(
        input_ids,
        max_length=20,
        num_beams=20,
        no_repeat_ngram_size=2,
        top_k=30,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=torch.ones_like(input_ids),
        do_sample=True
    )

    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

    print("Generated Poem:")
    print(generated_poem)

# Example :
stanza1 = 'بدو سرشبان گفت کای شهریار'
generate_poem(stanza1)


stanza2='جهاندار محمود خورشیدفش'
generate_poem(stanza2)

stanza3='و دیگر که دارنده یار منست'
generate_poem(stanza3)

<ipython-input-84-de5a7d789290>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_tokenized['input_ids']).to(device)


Generated Poem:
بدو سرشبان گفت کای شهریار    تو این خانه را خوارمایه مدار
Generated Poem:
جهاندار محمود خورشیدفش    هم اندر زمان دست کرده به کش
Generated Poem:
و دیگر که دارنده یار منست    روان و خرد تار و هم تنست
